In [19]:
import json
import pandas as pd
import re

# Directory containing the JSON files
directory = "C:/Users/tshan/Downloads/"

json_file_paths = [f"{directory}response ({i}).json" for i in range(1,6)]

all_employees_data = []

for file_path in json_file_paths:
    with open(file_path, 'r') as file:
        json_data = json.load(file)
        for employee in json_data.get("employees", []):
            profile = employee.get("profile", {})
            experiences = profile.get("experiences", []) or []
            education = profile.get("education", []) or []

            # Extract the most recent job title (if available)
            most_recent_job_title = "N/A"
            if experiences:
                most_recent_job_title = experiences[0].get("title", "N/A")

            # Extract all companies worked for
            worked_for_company = ", ".join(
                exp.get('company', 'N/A') for exp in experiences if exp.get('company')
            )

            # Define a priority list for academic hierarchy
            degree_priority = [
                "High School", "Associate", "Bachelor", "BA", "BS", "B.A", "B.A.", 
                "BE", "B.E", "BEng", "B.Eng","B.A.Sc.", "B.A.Sc",
                "B.Arch", "B.B.A", "B.S", "B.Sc", "B.Sc.", "LLM", 
                "Master", "Master's", "MPhil", "MA", "MS", "MSc", "MEng", 
                "JD", "J.D", "J.D.","PhD", "Ph.D"
            ]

            # Helper to read the year from degree['ends_at']
            def get_end_year(degree_item):
                """
                Returns the 'year' portion of degree_item['ends_at'], or None if missing.
                """
                ends_at = degree_item.get('ends_at', {})
                return ends_at.get('year') if isinstance(ends_at, dict) else None

            # Helper function: Additional heuristics for partial matching
            def get_degree_rank(degree_name):
                """Return an integer (or float) rank based on the known degree_priority list or heuristics.

                Lower rank => less advanced;
                Higher rank => more advanced.
                """
                degree_name = (degree_name or '').strip()

                def normalize_string(s):
                    # Make it lowercase and remove all non-alphanumeric characters
                    s = s.lower()
                    s = re.sub(r'[^a-z0-9]+', '', s)
                    return s

                degree_name_normalized = normalize_string(degree_name)

                # 1) Check for any known keywords in degree_priority (partial match)
                for idx, keyword in enumerate(degree_priority):
                    keyword_normalized = normalize_string(keyword)
                    if keyword_normalized in degree_name_normalized:
                        return idx

                # 2a) Advanced degree signals => treat as "PhD"
                if any(k in degree_name_normalized for k in ["doctor", "phd", "ph.d", "postdoc", "post-doc", "dphil","dma","md"]):
                    return degree_priority.index("PhD")

                # 2b) Bachelor-level signals => treat as "Bachelor"
                bachelor_candidates = [
                    "bachelor", "bachelor's", "beng", "undergrad", "undergraduate", 
                    "bsc", "ba", "be", "bs", "bba", "bfa", "barch", "basc","bm","btech","bsw"
                ]
                if any(k in degree_name_normalized for k in bachelor_candidates):
                    return degree_priority.index("Bachelor")

                # 2c) Master-level signals => treat as "Master"
                if any(k in degree_name_normalized 
                    for k in ["master", "mba", "mfa", "med", "mphil", "msc", "meng", "ma", "llm", "master's","mm", "msc"]):
                    return degree_priority.index("Master")

                # 2d) Less-advanced signals => rank below "Associate" but above "High School"
                if any(k in degree_name_normalized for k in [
                    "certificate", "exchange", "course", "short program", "abroad", "program", "language"
                ]):
                    associate_rank = degree_priority.index("Associate")
                    return associate_rank - 0.5  # e.g., 0.5 if "Associate" is index=1

                # 2e) Fallback => treat as more advanced than everything in the list
                return len(degree_priority)

            if education:
                # 1) Sort all education by ascending rank
                education_sorted_by_rank = sorted(
                    education, key=lambda x: get_degree_rank(x.get('degree_name', ''))
                )

                # 2) Determine first (least-advanced) among "Associate or above," 
                #    or fallback to the truly first if none are "Associate+"
                associate_index = degree_priority.index("Associate")
                valid_degrees = [
                    edu for edu in education_sorted_by_rank
                    if get_degree_rank(edu.get('degree_name', '')) >= associate_index
                ]
                if valid_degrees:
                    first_degree = valid_degrees[0]
                else:
                    first_degree = education_sorted_by_rank[0]

                first_degree_name = first_degree.get('degree_name', 'N/A')
                school_of_first_degree = first_degree.get('school', 'N/A')
                major_of_first_degree = first_degree.get('field_of_study', 'N/A')

                # 3) Default highest degree is the last item in ascending rank
                highest_degree = education_sorted_by_rank[-1]
                highest_degree_rank = get_degree_rank(highest_degree.get('degree_name', ''))

                # CHECK 1: PROTECT RECOGNIZED PHD FROM BEING OVERRIDDEN BY UNKNOWN
                phd_index = degree_priority.index("PhD")
                if highest_degree_rank > phd_index:
                    recognized_phds = [
                        edu for edu in education_sorted_by_rank
                        if get_degree_rank(edu.get('degree_name', '')) == phd_index
                    ]
                    if recognized_phds:
                        highest_degree = recognized_phds[-1]
                        highest_degree_rank = phd_index

                # CHECK 2: If current highest is unknown or < Associate
                #          but a known advanced degree ends later in time, pick that.
                highest_degree_rank = get_degree_rank(highest_degree.get('degree_name', ''))
                if (highest_degree_rank < associate_index) or (highest_degree_rank > phd_index):
                    unknown_end_year = get_end_year(highest_degree)
                    if unknown_end_year is not None:
                        advanced_degrees = [
                            edu for edu in education_sorted_by_rank
                            if get_degree_rank(edu.get('degree_name', '')) >= associate_index
                        ]
                        advanced_degrees_that_end_after = [
                            edu for edu in advanced_degrees
                            if (
                                get_end_year(edu) is not None 
                                and get_end_year(edu) > unknown_end_year
                            )
                        ]
                        if advanced_degrees_that_end_after:
                            highest_degree = advanced_degrees_that_end_after[-1]

                # Final highest_degree fields
                highest_degree_name = highest_degree.get('degree_name', 'N/A')
                school_of_highest_degree = highest_degree.get('school', 'N/A')
                major_of_highest_degree = highest_degree.get('field_of_study', 'N/A')
            else:
                # No education entries
                first_degree_name = "N/A"
                school_of_first_degree = "N/A"
                major_of_first_degree = "N/A"
                highest_degree_name = "N/A"
                school_of_highest_degree = "N/A"
                major_of_highest_degree = "N/A"

            # Convert experiences and education to string summaries
            experiences_summary = "; ".join(
                [
                    f"{exp.get('title', 'N/A')} at {exp.get('company', 'N/A')} "
                    f"({exp.get('starts_at', {}).get('year', 'N/A') if exp.get('starts_at') else 'N/A'} - "
                    f"{exp.get('ends_at', {}).get('year', 'N/A') if exp.get('ends_at') else 'N/A'})"
                    for exp in experiences if exp
                ]
            )
            education_summary = "; ".join(
                [
                    f"{edu.get('degree_name', 'N/A')} in {edu.get('field_of_study', 'N/A')} "
                    f"from {edu.get('school', 'N/A')} "
                    f"({edu.get('starts_at', {}).get('year', 'N/A') if edu.get('starts_at') else 'N/A'} - "
                    f"{edu.get('ends_at', {}).get('year', 'N/A') if edu.get('ends_at') else 'N/A'})"
                    for edu in education if edu
                ]
            )

            # Helper function to extract graduation year from profile
            def get_graduation_year(row):
                if row.get('education') and isinstance(row['education'], list):
                    graduation_years = [
                        edu.get('ends_at', {}).get('year')
                        for edu in row['education']
                        if edu.get('ends_at') and edu.get('ends_at').get('year')
                    ]
                    if graduation_years:
                        return max(graduation_years)
                if row.get('work_experience') and isinstance(row['work_experience'], list):
                    first_job_years = [
                        job.get('starts_at', {}).get('year')
                        for job in row['work_experience']
                        if job.get('starts_at') and job.get('starts_at').get('year')
                    ]
                    if first_job_years:
                        return min(first_job_years)
                return None

            graduation_year = get_graduation_year(profile)
            
            # Append processed data
            all_employees_data.append({
                "Company": "Imbue",
                "profile_url": employee.get("profile_url"),
                "last_updated": employee.get("last_updated"),
                "profile.full_name": profile.get("full_name"),
                "profile.occupation": profile.get("occupation"),
                "profile.headline": profile.get("headline"),
                "profile.country": profile.get("country"),
                "profile.city": profile.get("city"),
                "job_title": most_recent_job_title,
                "experiences_summary": experiences_summary,
                "education_summary": education_summary,
                "Highest Degree": highest_degree_name,
                "First Degree": first_degree_name,
                "School of First Degree": school_of_first_degree,
                "School of Highest Degree": school_of_highest_degree,
                "Major of First Degree": major_of_first_degree,
                "Major of Highest Degree": major_of_highest_degree,
                "Year of Graduation": graduation_year,
                "Year of Experience": 2024 - graduation_year if graduation_year else None,
                "Worked_for_company": worked_for_company,
            })

# Convert all aggregated data into a DataFrame
combined_df = pd.DataFrame(all_employees_data)

# Save the combined data to a CSV file
combined_output_file_path = "C:/Users/tshan/Downloads/Imbue.csv"
combined_df.to_csv(combined_output_file_path, index=False)

print("File saved as csv in the current directory.")


File saved as csv in the current directory.
